In [3]:
import pandas as pd
import os
import re

In [4]:
features_path = "/content/drive/MyDrive/Exercise Tracking/features"
combined_file = "/content/drive/MyDrive/Exercise Tracking/combined_features.csv"

def combine_features():
    combined_df = pd.DataFrame()
    for file in os.listdir(features_path):
        if file.endswith("_features.csv"):
            file_path = os.path.join(features_path, file)
            temp_df = pd.read_csv(file_path)

            exercise_class = re.sub(r'_\d+$', '', file.replace("_features.csv", ""))
            temp_df['class'] = exercise_class

            combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
    combined_df.to_csv(combined_file, index=False)
    print(f"Combined features saved to {combined_file}")

combine_features()

Combined features saved to /content/drive/MyDrive/Exercise Tracking/combined_features.csv


In [5]:
df = pd.read_csv(combined_file)

In [6]:
df.head()

,frame,distance_0_1,distance_0_2,distance_0_3,distance_0_4,distance_0_5,distance_0_6,distance_0_7,distance_0_8,distance_0_9,...,angle_8_9_11,angle_8_9_12,angle_8_10_11,angle_8_10_12,angle_8_11_12,angle_9_10_11,angle_9_10_12,angle_9_11_12,angle_10_11_12,class
0,frame_0000.jpg,0.948539,0.931362,0.948396,0.982577,0.974282,0.963229,0.950747,0.985354,0.826799,...,0.210850,0.119570,0.192963,0.129735,0.311227,0.435927,0.611312,0.436578,0.407620,Seated_Neck_Flexion
1,frame_0001.jpg,0.983620,0.959310,0.964760,0.936388,0.932630,0.929588,0.984134,0.959656,0.858508,...,0.212381,0.123711,0.188303,0.131701,0.300476,0.391939,0.562929,0.409395,0.382789,Seated_Neck_Flexion
2,frame_0002.jpg,0.983769,0.962355,0.961107,0.945847,0.939258,0.935513,0.995337,0.963668,0.856215,...,0.208364,0.122916,0.184636,0.131331,0.292878,0.399721,0.568098,0.396686,0.370590,Seated_Neck_Flexion
3,frame_0003.jpg,0.995239,0.971577,0.966162,0.942203,0.937052,0.935294,1.000000,0.954070,0.862612,...,0.209177,0.123941,0.183870,0.132111,0.288436,0.390387,0.557834,0.388214,0.362497,Seated_Neck_Flexion
4,frame_0004.jpg,0.946008,0.935534,0.936760,0.925012,0.931797,0.938595,0.973106,0.942545,0.877887,...,0.215448,0.128277,0.190416,0.136099,0.305247,0.394495,0.560515,0.404304,0.378765,Seated_Neck_Flexion


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8402 entries, 0 to 8401
Columns: 366 entries, frame to class
dtypes: float64(364), object(2)
memory usage: 23.5+ MB


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df.drop(columns=['frame', 'class'])
y = df['class']

In [10]:
y.value_counts()

,count
class,
Neck_Rotation,1124
Seated_Neck_Stretch,1104
Seated_Head_Turns,1046
Neck_Side_Bend,1010
Neck_Isometrics,778
Seated_Neck_Flexion,729
Neck_Flexion,728
Shoulder_Shrugs,675
Neck_Extension,624


In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [13]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
cv = cross_val_score(rf_model, X_train, y_train, cv=5)
print(f"Cross-Validation Scores: {cv}")
print(f"Mean CV Score: {cv.mean()}")

Cross-Validation Scores: [0.96208178 0.96428571 0.95833333 0.96056548 0.95610119]
Mean CV Score: 0.9602734997344664


In [15]:
#predict on validation set
y_pred = rf_model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

Accuracy: 0.9678571428571429
Classification Report:
                     precision    recall  f1-score   support

         Chin_Tucks       0.98      1.00      0.99        50
     Neck_Extension       1.00      0.89      0.94        66
       Neck_Flexion       1.00      0.95      0.98        65
    Neck_Isometrics       1.00      0.96      0.98        79
      Neck_Rotation       0.95      0.98      0.97       116
     Neck_Side_Bend       1.00      0.95      0.98       111
  Seated_Head_Turns       0.98      0.96      0.97       100
Seated_Neck_Flexion       0.91      0.99      0.95        75
Seated_Neck_Stretch       0.92      0.99      0.96       108
    Shoulder_Shrugs       0.97      0.99      0.98        70

           accuracy                           0.97       840
          macro avg       0.97      0.97      0.97       840
       weighted avg       0.97      0.97      0.97       840

Confusion Matrix:
[[ 50   0   0   0   0   0   0   0   0   0]
 [  0  59   0   0   0   0   0 

In [16]:
# Predict on the test set
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9667063020214031
Classification Report:
                     precision    recall  f1-score   support

         Chin_Tucks       0.97      0.94      0.95        62
     Neck_Extension       0.98      0.96      0.97        52
       Neck_Flexion       0.98      0.99      0.98        86
    Neck_Isometrics       0.99      0.99      0.99        70
      Neck_Rotation       0.96      0.98      0.97       103
     Neck_Side_Bend       0.96      0.92      0.94       106
  Seated_Head_Turns       0.98      0.96      0.97       110
Seated_Neck_Flexion       0.96      0.97      0.97        80
Seated_Neck_Stretch       0.93      0.96      0.94       107
    Shoulder_Shrugs       0.98      1.00      0.99        65

           accuracy                           0.97       841
          macro avg       0.97      0.97      0.97       841
       weighted avg       0.97      0.97      0.97       841

Confusion Matrix:
[[ 58   0   1   1   0   0   0   2   0   0]
 [  1  50   1   0   0   0   0 

In [19]:
import joblib

joblib.dump(rf_model, '/content/drive/MyDrive/Exercise Tracking/exercise_classifier.pkl')

['/content/drive/MyDrive/Exercise Tracking/exercise_classifier.pkl']